In [1]:
import pandas as pd
import bpy
url = 'https://simplemaps.com/static/data/country-cities/de/de.csv'
df = pd.read_csv(url)
# Drop rows where population_proper is NaN
df = df.dropna(subset=['population_proper'])
df

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Berlin,52.5200,13.4050,Germany,DE,Berlin,primary,4890363.0,3755251.0
1,Stuttgart,48.7775,9.1800,Germany,DE,Baden-Württemberg,admin,2787724.0,632865.0
2,Munich,48.1375,11.5750,Germany,DE,Bavaria,admin,2606021.0,1512491.0
3,Hamburg,53.5500,10.0000,Germany,DE,Hamburg,admin,2484800.0,1892122.0
4,Cologne,50.9364,6.9528,Germany,DE,North Rhine-Westphalia,NaN,1084831.0,1084831.0
...,...,...,...,...,...,...,...,...,...
122,Neulußheim,49.2933,8.5219,Germany,DE,Baden-Württemberg,NaN,7128.0,7128.0
123,Eichwalde,52.3667,13.6167,Germany,DE,Brandenburg,NaN,6475.0,6475.0
124,Altbach,48.7239,9.3797,Germany,DE,Baden-Württemberg,NaN,6422.0,6422.0
125,Merzhausen,47.9664,7.8286,Germany,DE,Baden-Württemberg,NaN,5347.0,5347.0


In [1]:
!uv pip install bpy-pandas-mesh

Using Python 3.11.10 environment at /Users/jan-hendrik/Desktop/blender_python_workshop/.venv
Audited 1 package in 3ms


In [2]:
from pandas_mesh import PandasMesh

In [4]:
blender_mesh = PandasMesh(dataframe=df, object_name="PopulationMesh")

# Select the object by name
bpy.data.objects["PopulationMesh"].select_set(True)
bpy.context.view_layer.objects.active = bpy.data.objects["PopulationMesh"]

Columns not included (unsupported data types): ['city', 'country', 'iso2', 'admin_name', 'capital']
Columns added to the mesh: ['lat', 'lng', 'population', 'population_proper']


In [5]:
df_adjusted = df.copy()
df_adjusted['lat'] = df_adjusted['lat'] - 50
df_adjusted['lng'] = df_adjusted['lng'] - 8
df_adjusted['population_proper'] = df_adjusted['population_proper'] * 0.0000005

blender_mesh.update(dataframe=df_adjusted)

Columns not included (unsupported data types): ['city', 'country', 'iso2', 'admin_name', 'capital']
Columns added to the mesh: ['lat', 'lng', 'population', 'population_proper']


In [ ]:
import pandas as pd

# Example dataframe with various data types
data = {
    'height': [1.5, 1.7, 1.6],
    'weight': [50, 60, 55],
    'age': [25, 30, 28],
    'name': ['Alice', 'Bob', 'Charlie'],  # String column (unsupported)
    'position': [(1, 2, 3), (4, 5, 6), (7, 8, 9)],  # Vector data
    'color': [(0.1, 0.2, 0.3, 0.4), (0.5, 0.6, 0.7, 0.8), (0.9, 1.0, 0.8, 0.7)],  # Color data
    'is_active': [True, False, True]  # Boolean data
}
df2 = pd.DataFrame(data)

# Create a mesh with the dataframe
mesh = PandasMesh(
    dataframe=df2,
    mesh_name="PersonAttributesMesh",
    object_name="PersonAttributesObject"
)

# Expected Output:
# Columns not included (unsupported data types): ['name']
# Columns added to the mesh: ['height', 'weight', 'age', 'position', 'color', 'is_active']

In [ ]:
print(df2)

In [ ]:
north = 55.1  # Northern latitude in deg
south = 47.2  # Southern latitude in deg
west = 5.5    # Western longitude in deg
east = 15.5   # Eastern longitude in deg
offset_x = -8
offset_y = -50
# Calculate the center coordinates of the bounding box
center_x = (west + east) / 2
center_y = (north + south) / 2

map_location = (center_x + offset_x, center_y + offset_y, 0 )
# Add the plane and set it at the calculated center location
bpy.ops.mesh.primitive_plane_add(size=1, location= map_location)
plane = bpy.context.object

# Create a new material and assign the image texture
material = bpy.data.materials.new(name="ImageMaterial")
material.use_nodes = True
bsdf = material.node_tree.nodes["Principled BSDF"]

# Add and load the image texture
tex_image = material.node_tree.nodes.new('ShaderNodeTexImage')
tex_image.image = bpy.data.images.load("Germany_location_map.png")

# Connect the texture to the Base Color of the BSDF shader
material.node_tree.links.new(bsdf.inputs['Base Color'], tex_image.outputs['Color'])
plane.data.materials.append(material)

# Calculate the scale based on the geographic bounds
plane.scale.x = (east - west)
plane.scale.y = (north - south) 